# UNSW-NB15 Feature Importance

https://www.kaggle.com/khairulislam/unsw-nb15-feature-importance

## Imports

In [ ]:
import warnings
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Functions

In [ ]:
# 1. Reading Train and test dataset.
# 2. Check if dataset is reversed.
# 3. Drop 'id', and 'attack_cat' columns.
def input_train_test():
    train = pd.read_csv('UNSW_NB15_training-set.csv')
    test = pd.read_csv('UNSW_NB15_testing-set.csv')
    
    if train.shape < test.shape:
        # Reversing the dataset
        train, test = test, train
        # Dropping the columns
        drop_columns = ["id", "attack_cat"]
        print("Dropped 'id', and 'attack_cat' columns")
        train.drop(drop_columns, axis=1, inplace=True)
        test.drop(drop_columns, axis=1, inplace=True)
        print("Train and Test sets are reversed, Corrected Shape:")
        print("Train shape: ", train.shape)
        print("Test shape: ", test.shape)
    else:
        print("The dataset, is already reversed")
        print("Train shape: ", train.shape)
        print("Test shape: ", test.shape)
    return train, test

In [ ]:
# identifying object columns and appending them inside cat_cols[]
# The training and testing datasets have identical sets of columns. 
# Fine with getting the column names from only on of the subsets.
def get_cat_cols(train):
    # Defining an empty list
    categorical = []
    # Iterating through the columns and checking for columns with datatyp "Object"
    for col in train.columns:
        if train[col].dtype == 'object':
            categorical.append(col) # appending "object" columns to cat_cols
    return categorical

In [ ]:
def label_encode(train, test):
    for col in get_cat_cols(train):
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))
    return train, test

In [ ]:
def feature_engineer(df):
    df.loc[~df['state'].isin(['FIN', 'INT', 'CON', 'REQ', 'RST']), 'state'] = 'others'
    df.loc[~df['service'].isin(['-', 'dns', 'http', 'smtp', 'ftp-data', 'ftp', 'ssh', 'pop3']), 'service'] = 'others'
    df.loc[df['proto'].isin(['igmp', 'icmp', 'rtp']), 'proto'] = 'igmp_icmp_rtp'
    df.loc[~df['proto'].isin(['tcp', 'udp', 'arp', 'ospf', 'igmp_icmp_rtp']), 'proto'] = 'others'
    return df

In [ ]:
def get_train_test(train, test, label_encoding=False, scaler=None):
    x_train, y_train = train.drop(['label'], axis=1), train['label']
    x_test, y_test = test.drop(['label'], axis=1), test['label']
    
    # Running x_train and x_test (the inputs) into feature engineering.
    x_train, x_test = feature_engineer(x_train), feature_engineer(x_test)
    
    categorical_columns = get_cat_cols(x_train)
    non_categorical_columns = [x for x in x_train.columns if x not in categorical_columns]   
    if scaler is not None:
        x_train[non_categorical_columns] = scaler.fit_transform(x_train[non_categorical_columns])
        x_test[non_categorical_columns] = scaler.transform(x_test[non_categorical_columns])
    
    if label_encoding:
        x_train, x_test = label_encode(x_train, x_test)
        features = x_train.columns
    else:
        x_train = pd.get_dummies(x_train)
        x_test = pd.get_dummies(x_test)
        print("Column mismatch {0}, {1}".format(set(x_train.columns)- set(x_test.columns),  set(x_test.columns)- set(x_train.columns)))
        features = list(set(x_train.columns) & set(x_test.columns))
    print(f"Number of features {len(features)}")
    x_train = x_train[features]
    x_test = x_test[features]
    
    return x_train, y_train, x_test, y_test

In [ ]:
def show_feature_importance(importance, columns):
    feature_importance = pd.DataFrame(zip(columns, importance), columns=['Feature', 'Importance'])
    feature_importance['Importance'] /= feature_importance['Importance'].sum()*0.01
    return feature_importance.sort_values(by="Importance", ascending=False)

## Preperation of the Data

In [ ]:
train, test = input_train_test()
categorical_columns = get_cat_cols(train)

In [ ]:
folds = 10
seed = 1
num_round = 2000
kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
X, Y, x_test, y_test = get_train_test(train, test, label_encoding=True, scaler= StandardScaler())
importance_dict = {
    "feature": X.columns
}

In [ ]:
clf = RandomForestClassifier(random_state=1)
clf.fit(X, Y)
feature_importance = clf.feature_importances_
importance_dict['train'] =  feature_importance
# show_feature_importance(feature_importance, X.columns)

In [19]:
feature_importances = []

for tr_idx, val_idx in tqdm(kf.split(X, Y), total=folds):
    x_train, y_train = X.iloc[tr_idx], Y[tr_idx]
    # x_val, y_val = X.iloc[val_idx], Y[val_idx]
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    
    feature_importances.append(clf.feature_importances_)

feature_importance = np.mean(feature_importances, axis=0)
importance_dict['train_10_fold'] =  feature_importance
# show_feature_importance(feature_importance, X.columns)

C:\Users\Sajepan\AppData\Local\Temp/ipykernel_16544/3138433173.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for tr_idx, val_idx in tqdm(kf.split(X, Y), total=folds):


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html